In [ ]:
%load_ext autoreload
%autoreload 2
from aggregated_models.myimports  import *
# import aggregated_models.myJupyterUtils as myJupyterUtils ## Remove stacktraces on Keyboardinterupt
plt.style.use('ggplot')
import getpass

# helpers to compute metrics
from aggregated_models.validation import SparkMetricsComputer

# baselines

# loading public "criteo attribution dataset"

In [ ]:
%matplotlib inline

In [ ]:
# code to prepare the aggregated dataset
from aggregated_models.aggdataset import AggDataset 


In [ ]:
## Most relevant code is there:
from aggregated_models.agg_mrf_model import AggMRFModel, AggMRFModelParams
# also in https://gitlab.criteois.com/a.gilotte/aggdata/-/blob/master/src/baseaggmodel.py

In [ ]:
from thx.hadoop.spark_config_builder import create_remote_spark_session
from thx.datasources.parquet import create_df_from_parquet
from datetime import datetime

In [ ]:
ss = create_remote_spark_session('Test Spark parallelize', 200, 8, memory='4g', memoryOverhead='2g', driver_memory='12g', hadoop_file_systems=['viewfs://root', 'viewfs://prod-am6'])
ss

## Loading data
3 versions of the dataset are used for experiments: "small" , "sampled" and "full"
- "full" has 11 features with about 16M samples
- "sampled" has the same 11 features, but only 160k samples
- "small" also has 160k samples, but only the 5 features with lowest modalities count, and allow for fast experiments.

In [ ]:
# dataset= "small_tb" # fast expriments
# dataset= "medium_tb" # fast expriments
# dataset= "sampled" # fast expriments
# dataset= "small" # fast expriments
# dataset= "sampled" # Training a MRF may require 5h and 16Go data
# dataset= "full"  # Training a MRF may require 32Go, and several days

In [ ]:
start_date = datetime(2015, 2, 15)
validation_split_date = datetime(2015, 3,3)
end_date = datetime(2015, 3, 11)

In [ ]:
train = create_df_from_parquet(
            ss,
            'viewfs://prod-am6/user/testfwk/datasets/criteo_1tb_parquet/date={year:04}-{month:02}-{day:02}',
            None,
            start_date,
            validation_split_date
        )

In [ ]:
valid = create_df_from_parquet(
            ss,
            'viewfs://prod-am6/user/testfwk/datasets/criteo_1tb_parquet/date={year:04}-{month:02}-{day:02}',
            None,
            validation_split_date,
            end_date
        )

In [ ]:
train.columns

In [ ]:
features = [c for c in train.columns if 'feature' in c][0:13]

In [ ]:
features

In [ ]:
label = 'label'

## Preparing Aggregated data

- aggdata contains projections of number of displays and clicks along each pair of feature
- may also add some noise to make it differential private
- the goal is to learn a model predicting Proba( label | features) using *only* those aggdata.

In [ ]:
aggdata = AggDataset(dataframe=train, features=features, label=label, maxNbModalities=10000)

In [ ]:
aggdata

In [ ]:
Validation = SparkMetricsComputer(label,"prediction")

In [ ]:
# parameters for of the privacy protecting noise.
epsilon = None  # Set to None to get no noise.
delta = None 

##### Proposed MRF model
- uses only aggregated data
- almost retrieves logitic performances

In [ ]:
regulL2 = 16
nbSamples = 10000
nbIter = 100

In [ ]:
aggdata

In [ ]:
ss.sparkContext.setCheckpointDir(f"viewfs://prod-am6/tmp/{getpass.getuser()}/load/checkpoints")

In [ ]:
params = AggMRFModelParams(
    exactComputation=False ,  ## Using Gibbs Sampling.  actualy exact=True is broken in latest code
    clicksCfs = "*&*", ## crossfeatures used by P(Y|X) part of the model
    displaysCfs="*&*", ## crossfeatures used by P(X) part of the model. Here, all pairs + all single .
    nbSamples = nbSamples, ## Nb Gibbs samples to estimate gradient
    regulL2=1.0 ,  ## parmeter "lambda_2"
    regulL2Click = regulL2,  ## parmeter "lambda_1"
    sampleFromPY0 = True,
    maxNbRowsperGibbsUpdate = 1,
)
rddMrf = AggMRFModel(aggdata, features, params, sparkSession = ss)

In [ ]:
rddMrf.fit(1, 0.05)

In [ ]:
print( f"MRF" ,  "train",  Validation.run(rddMrf, train) , "valid" , Validation.run(rddMrf,valid)   )